We import `netgen.gui` to open the netgen GUI for displaying the ferromagnet. 
We then define the geometry, and draw the box. 
The refinement of the mesh is controlled by `H_MAX` in `ngmesh`.

In [8]:
from netgen.csg import *
from ngsolve import *
from ngsolve.utils import grad, Grad  # If I don't import these explicitly, VSCode reads them as missing.
import netgen.gui  # this opens up the netgen ui
import Magnetisation_Functions as magfunc
import Elastic_Functions as elfunc
import numpy as np
import scipy.sparse as sp
T_MAX = 10.0  # The maximum time for the simulation
ALPHA = 1  # Dissipative constant in the LLG equation.
THETA = 0.75 # Should be strictly above 1/2 for unconditional stability
K = 0.1  # TIME STEP
KAPPA = 1  # Determines the relative strength of the elastic vs. magnetic parts.
H_MAX = 1  # Determines how fine the mesh should be.

In [9]:
def MakeGeometry():  # this makes a box, with labelled faces
    geometry = CSGeometry()
    left  = Plane(Pnt(0,0,0), Vec(-1,0,0)).bc("left")
    right = Plane(Pnt(1,1,1), Vec( 1,0,0)).bc("right")
    front = Plane(Pnt(0,0,0), Vec(0,-1,0)).bc("front")
    back  = Plane(Pnt(1,1,1), Vec(0, 1,0)).bc("back")
    bot   = Plane(Pnt(0,0,0), Vec(0,0,-1)).bc("bot")
    top   = Plane(Pnt(1,1,1), Vec(0,0, 1)).bc("top")

    cube = left * right * front * back * bot * top
    geometry.Add (cube)
    #cube = OrthoBrick(Pnt(0,0,0), Pnt(1,1,1))
    geometry.Add(cube)
    return geometry



ngmesh = MakeGeometry().GenerateMesh(maxh=H_MAX)
# ngmesh.Save("cube.vol")
mesh = Mesh(ngmesh)
Draw(mesh)

In [10]:
  # this helps me figure out which sides are which.
#fes_SCALAR = H1(mesh, order=1)
#myScalars = {"left":10, "right":10, "front":0, "back":0, "bot":5, "top":5}
#SCALAR_gfu = CoefficientFunction([myScalars[val] for val in mesh.GetBoundaries()])

#Draw(SCALAR_gfu, mesh, "SCALAR_gfu")

In [11]:
fes_mag = VectorH1(mesh, order=1)  # the finite element space for the magnetisation m_h^i
fes_disp = VectorH1(mesh, order=1, dirichlet="bot")  # the finite element space for the displacement u_h^i

print(f"mag_ndof={fes_mag.ndof}, disp_ndof={fes_disp.ndof},\n, dispfree_ndof={fes_disp.FreeDofs()}")
mag_gfu = GridFunction(fes_mag)
disp_gfu = GridFunction(fes_disp)

mag_ndof=27, disp_ndof=27,
, dispfree_ndof=0: 010011011010011011010011011


In [12]:
v = fes_mag.TrialFunction()
phi = fes_mag.TestFunction()
u = fes_disp.TrialFunction()
psi = fes_disp.TestFunction()

mag_gfu = magfunc.give_random_magnetisation(mag_gfu)

a_mag = BilinearForm(fes_mag)
a_mag += ALPHA*InnerProduct(v, phi)*dx
a_mag += THETA*K*InnerProduct(Grad(v), Grad(phi))*dx
a_mag += InnerProduct(Cross(mag_gfu,v), phi)*dx
a_mag.Assemble()

f_mag = LinearForm(fes_mag)
f_mag += -InnerProduct(Grad(mag_gfu), Grad(phi))*dx
#f_mag +=  # This is where we will put the magnetostrictive contribution.
f_mag.Assemble()

a_disp = BilinearForm(fes_disp)
a_disp += InnerProduct(u, psi)*dx
a_disp += InnerProduct(elfunc.stress)*dx

In [15]:
B_T = magfunc.build_tangent_plane_matrix_transpose(mag_gfu)
type(a_mag), type(B_T), type(f_mag)

(ngsolve.comp.BilinearForm, ngsolve.bla.MatrixD, ngsolve.comp.LinearForm)

In [13]:
with np.printoptions(threshold=np.inf):
    print(magfunc.give_magnetisation_update(a_mag, B_T, f_mag))

[-0.33187282 -0.01700777 -0.69531442  0.15584503 -0.01050842 -0.85927957
 -0.45900901 -0.48776918  2.70491617 -0.94726553 -0.86653848 -0.47129851
 -0.47533919  0.14280275 -0.09125148 -0.75157839 -0.30934734  3.76981617
  0.01275178  0.06971     0.67224149  0.33809674  0.8316015   0.2569797
 -0.1357197  -0.33507428 -1.71058723  0.          0.          0.
  0.          0.          0.          0.          0.          0.        ]


In [14]:
#print(mag_gfu.vec)
#Draw(mag_gfu)
#print(magfunc.build_tangent_plane_matrix_transpose(mag_gfu))
